In [34]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [121]:
import numpy as np
import open3d as o3d
import copy

from utils import readXML, Intrinsics, Extrinsics, RGBD, crop_geometry, draw_registration_result, pick_points, manual_registration

In [126]:
xml_dir = "/home/yunzhu/Documents/project_phystouch/phystouch/data_recording/data"
xml_dir = "C:/Users/lucas/Desktop/UPC/MIT/tactile2object/intrinsics_extrinsics/data/"

intrinsics = Intrinsics(xml_dir)
extrinsics = Extrinsics(xml_dir)
rgbds = RGBD(xml_dir, recording="recording/20210709_004128")

# Stitching point clouds

### camera 020122061233

In [146]:
camera = "020122061233"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_1 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)

#o3d.visualization.draw_geometries([pcd_1])

### camera 821312060044

In [147]:
camera = "821312060044"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_2 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)


### camera 020122061651

In [148]:
camera = "020122061651"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_3 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)


### camera 821312062243

In [149]:
camera = "821312062243"

rgbd = rgbds.from_camera(camera)
intrinsic = intrinsics.from_camera(camera)
extrinsic = extrinsics.from_camera(camera)

pcd_4 = o3d.geometry.PointCloud.create_from_rgbd_image(rgbd, intrinsic, extrinsic)


### Stitched Point Cloud

In [150]:
pcds = pcd_1 + pcd_2 + pcd_3 + pcd_4

o3d.visualization.draw_geometries([pcds])

# ICP

In [10]:
obj_name = "005_tomato_soup_can"
obj_name = "003_cracker_box"

obj_path = f"C:/Users/lucas/Desktop/UPC/MIT/tactile2object/estimate_object_pose/ycb_converted/{obj_name}.ply"

obj_pcd = o3d.io.read_point_cloud(obj_path)
obj_mesh = o3d.io.read_triangle_mesh(obj_path)

obj_pcd.scale(0.001, center=(0,0,0))
obj_mesh.scale(0.001, center=(0,0,0))

TriangleMesh with 8194 points and 16384 triangles.

In [11]:
source = copy.deepcopy(obj_pcd)
target = copy.deepcopy(pcds)
transformation = manual_registration(source, target)

Manual ICP
Visualization of two point clouds before manual alignment
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 

1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #4541 (0.013, -0.093, 0.21) to add in queue.
[Open3D INFO] No point has been picked.
[Open3D INFO] Picked point #8172 (0.015, 0.064, 0.21) to add in queue.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 
[Open3D INFO] Picked point #2281 (0.016, 0.064, 0.0027) to add in queue.
[Open3D INFO] Picked point #1043 (0.017, -0.094, -8.7e-05) to add in queue.


1) Please pick at least three correspondences using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked

In [12]:
obj_mesh_t = copy.deepcopy(obj_mesh).transform(transformation)

obj_mesh_t.compute_vertex_normals()
obj_mesh_t.paint_uniform_color([1, 0.706, 0])

o3d.visualization.draw_geometries([pcds, obj_mesh_t])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 
